In [ ]:
# Import required librariesimport jsonimport osfrom typing import Dict, Listfrom dotenv import load_dotenvfrom azure.identity import DefaultAzureCredential, AzureCliCredentialfrom azure.search.documents import SearchClientimport autogenfrom autogen import AssistantAgent, UserProxyAgent, register_function

In [ ]:
# Load environment variablesload_dotenv()AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")AZURE_SEARCH_API_VERSION = os.getenv("AZURE_SEARCH_API_VERSION")AZURE_SEARCH_SEMANTIC_CONFIG = os.getenv("AZURE_SEARCH_SEMANTIC_SEARCH_CONFIG")AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

In [ ]:
# Check Azure CLI installationprint("Checking Azure CLI installation...")try:    import subprocess    subprocess.run(["az", "--version"], check=True)    print("Azure CLI is installed and working.")except Exception as e:    print(f"Error: {e}. Please install Azure CLI.")

In [ ]:
# Authenticate and create SearchClientprint("Authenticating with Azure...")credential = AzureCliCredential()endpoint = AZURE_SEARCH_SERVICE_ENDPOINTclient = SearchClient(endpoint=endpoint, index_name=AZURE_SEARCH_INDEX, credential=credential)

In [ ]:
# Define search tool functiondef search(query: str) -> List[Dict]:    """    Searches the Azure Cognitive Search index for the given query.    Returns a list of dictionaries containing search results.    """    payload = json.dumps({        "search": query,        "vectorQueries": [{"kind": "text", "text": query, "k": 5, "fields": "vector"}],        "queryType": "semantic",        "semanticConfiguration": AZURE_SEARCH_SEMANTIC_CONFIG,        "captions": "extractive",        "answers": "extractive|count-3",        "queryLanguage": "en-US",    })    response = list(client.search(payload))    output = []    for result in response:        result.pop("titleVector", None)        result.pop("contentVector", None)        output.append(result)    return output

In [ ]:
# Configure assistant agentconfig_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")gpt4_config = {    "cache_seed": 42,    "temperature": 0,    "config_list": config_list,    "timeout": 120,}

In [ ]:
# Create assistant and user agentscog_search = AssistantAgent(    name="COGSearch",    system_message="You are a helpful AI assistant. You can help with Azure Cognitive Search. Return 'TERMINATE' when the task is done.",    llm_config=gpt4_config,)user_proxy = UserProxyAgent(    name="User",    llm_config=False,    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],    human_input_mode="NEVER",)

In [ ]:
# Register search tool with agentsregister_function(    search,    caller=cog_search,    executor=user_proxy,    name="search",    description="A tool for searching the Cognitive Search index",)

In [ ]:
# Initiate chatasync def main():    with autogen.Cache.disk() as cache:        await user_proxy.a_initiate_chat(            cog_search,            message="Search for 'What is Azure?' in the Cognitive Search index",            cache=cache,        )

In [ ]:
if __name__ == "__main__":    import asyncio    asyncio.run(main())